In [1]:
import datetime, re
import googleapiclient.discovery
import google.auth

In [2]:
from dateutil.parser import parse as dtparse
from datetime import datetime as dt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/module3/CalendarApiQueries

/content/gdrive/My Drive/module3/CalendarApiQueries


In [5]:
%pwd

'/content/gdrive/My Drive/module3/CalendarApiQueries'

In [6]:
# Preparation for Google API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
calendar_id = 'arnold@dsi-program.com'
gapi_creds = google.auth.load_credentials_from_file('credentials.json', SCOPES)[0]
service = googleapiclient.discovery.build('calendar', 'v3', credentials=gapi_creds)

In [7]:
# Get events from Google Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z'
events_result = service.events().list(
     calendarId=calendar_id, timeMin=now,
     maxResults=5, singleEvents=True,
     orderBy='startTime').execute()
events_result

{'accessRole': 'reader',
 'defaultReminders': [],
 'etag': '"p32ge1umhn7ufc0g"',
 'items': [{'attendees': [{'email': 'edoardo@dsi-program.com',
     'responseStatus': 'accepted'},
    {'email': 'antsa@dsi-program.com',
     'organizer': True,
     'responseStatus': 'accepted'},
    {'email': 'arnold@dsi-program.com',
     'responseStatus': 'accepted',
     'self': True},
    {'email': 'mulricharmel@gmail.com', 'responseStatus': 'needsAction'},
    {'email': 'nadeem@dsi-program.com', 'responseStatus': 'needsAction'}],
   'conferenceData': {'conferenceId': 'gkr-rzbr-vyx',
    'conferenceSolution': {'iconUri': 'https://fonts.gstatic.com/s/i/productlogos/meet_2020q4/v6/web-512dp/logo_meet_2020q4_color_2x_web_512dp.png',
     'key': {'type': 'hangoutsMeet'},
     'name': 'Google Meet'},
    'entryPoints': [{'entryPointType': 'video',
      'label': 'meet.google.com/gkr-rzbr-vyx',
      'uri': 'https://meet.google.com/gkr-rzbr-vyx'},
     {'entryPointType': 'more',
      'pin': '633281700834

In [8]:
# Pick up only start time, end time and summary info
events = events_result.get('items', [])
formatted_events = [(event['start'].get('dateTime', event['start'].get('date')), # start time or day
     event['end'].get('dateTime', event['end'].get('date')), # end time or day
     event['summary']) for event in events]
#formatted_events
for event in formatted_events:
  print(event, "\n")

('2022-04-06T13:00:00+02:00', '2022-04-06T14:00:00+02:00', 'DSI Module 3 Stand-up') 

('2022-04-06T17:00:00+02:00', '2022-04-06T18:00:00+02:00', 'DSI Seminar: Dr Sudeep Das (Netflix) - RL for recommender systems') 

('2022-04-07T13:00:00+02:00', '2022-04-07T14:00:00+02:00', 'DSI Module 3 Stand-up') 

('2022-04-08T13:00:00+02:00', '2022-04-08T14:00:00+02:00', 'DSI Module 3 Stand-up') 

('2022-04-09T13:00:00+02:00', '2022-04-09T14:00:00+02:00', 'DSI Module 3 Stand-up') 



### MORE NLP Queries (Antsa)

##### What is the next task of the day

In [20]:
# Next event
try:
  now = datetime.datetime.utcnow().isoformat() + 'Z'
  events_result = service.events().list(
      calendarId=calendar_id, timeMin=now,
      maxResults=1, singleEvents=True,
      orderBy='startTime').execute()

  events = events_result.get('items', [])
  #tmfmt = '%d %B, %H:%M %p'
  tf = '%H:%M %p'
  response = [(event['summary'], 
              event['start'].get('dateTime', event['start'].get('date')), 
              event['end'].get('dateTime', event['end'].get('date'))) for event in events][0]
              
  print("Your next meeting is", response[0], ". It starts at", dt.strftime(dtparse(response[1]), format=tf))
except:
  print("You dont have meetings left today")

# formatted_events = [(event['start'].get('dateTime', event['start'].get('date')), # start time or day
#      event['end'].get('dateTime', event['end'].get('date')), # end time or day
#      event['summary']) for event in events]

Your next meeting is DSI Module 3 Stand-up . It starts at 13:00 PM


##### What meetings do i have tomorrow

In [19]:
tomorrow_start = (datetime.date.today() + datetime.timedelta(days=1)).isoformat() + 'T00:00:0.0Z'
tomorrow_end = (datetime.date.today() + datetime.timedelta(days=1)).isoformat() + 'T23:59:0.0Z'

events_result = service.events().list(
     calendarId=calendar_id, timeMin=tomorrow_start, timeMax=tomorrow_end,
     maxResults=10, singleEvents=True,
     orderBy='startTime').execute()

events = events_result.get('items', [])

tf = '%H:%M %p'
response = [(event['summary'], 
             event['start'].get('dateTime', event['start'].get('date')), 
             event['end'].get('dateTime', event['end'].get('date'))) for event in events]

print("Your tomorow's events are as follows: ", [(response[0][0],'at', dt.strftime(dtparse(response[0][1]), format=tf)) for item in response])

Your tomorow's events are as follows:  [('DSI Module 3 Stand-up', 'at', '13:00 PM')]


###### How many events do i have next week

In [26]:
one_day = datetime.timedelta(days=1)

def get_week(date):
  """Return the full week (Sunday first) of the week containing the given date.

  'date' may be a datetime or date instance (the same type is returned).
  """
  day_idx = (date.weekday() + 1) % 7  # turn sunday into 0, monday into 1, etc.
  sunday = date - datetime.timedelta(days=day_idx)
  date = sunday
  for n in range(7):
    yield date
    date += one_day

this_week = [d.isoformat() for d in get_week(datetime.datetime.now().date())]

week_start_date = this_week[0] + 'T00:00:0.0Z'
week_end_date = this_week[-1] + 'T23:59:0.0Z'

events_result = service.events().list(
     calendarId=calendar_id, timeMin=week_start_date, timeMax=week_end_date,
     maxResults=100, singleEvents=True,
     orderBy='startTime').execute()

events = events_result.get('items', [])
print('You have ', len(events), 'this week.')

You have  11 this week.
